# Tesseract test

In [ ]:
import os, sys
import nltk

import numpy as np

# Change directory
os.getcwd().split('/')[-1] == "experimentation" and os.chdir('../')
    

# Import the full model
from src.text_processor import *
from src.text_postprocessor import *

from src.full_model import FullModel

In [2]:
def jaccard_similarity(t1, t2):
    """
    Calculates the Jaccard similarity between two texts.
    
    Parameters:
    text1 (str): First text.
    text2 (str): Second text.
    
    Returns:
    float: Jaccard similarity between the two texts (value between 0 and 1).
    """
    # Convert texts into sets of words
    set1 = set(t1.lower().split())
    set2 = set(t2.lower().split())
    
    # Calculate intersection and union of the sets
    intersection = set1.intersection(set2)
    union = set1.union(set2)
    
    # Calculate Jaccard similarity
    if len(union) == 0:
        return 0.0
    else:
        return len(intersection) / len(union)

In [3]:
def cer_similarity(t1, t2):
    """
    Calculates the Character Error Rate (CER) between two texts.
    
    Parameters:
    text1 (str): First text.
    text2 (str): Second text.
    
    Returns:
    float: CER between the two texts (value between 0 and 1).
    """
    # Calculate the edit distance
    dist = nltk.edit_distance(t1, t2)
    
    # Calculate the CER
    if len(t1) == 0 and len(t2) == 0:
        return 0.0
    else:
        return dist / max(len(t1), len(t2))

In [4]:
def read_text(path):
    
    with open(path, "r") as file:
        text = file.read()
        return text

In [5]:
import re

def normalize_text(text):
    
    palabras = re.findall(r'\b[a-zA-Z]+\b', text.lower())
    return ' '.join(palabras)

In [ ]:
precision = 3
crops_path = "./dataset/crops"

directory = [i.split('.')[-2] for i in os.listdir(f"{crops_path}/images")]

print(directory)

images = [f"{crops_path}/images/{item}.png" for item in directory]
texts = [read_text(f"{crops_path}/texts/{item}.txt") for item in directory]

texts = [normalize_text(text) for text in texts]

results = parametric_search(images)

len0 = len(results)
len1 = len(results[0])

mt_jaccard = np.zeros((len0, len1))
mt_cer = np.zeros((len0, len1))

for i in range(len(results)):
    
    real_text = texts[i]
    
    for j in range(len(results[0])):
        
        results[i][j] = normalize_text(results[i][j])
        
        mt_jaccard[i, j] = jaccard_similarity(real_text, results[i][j])
        mt_cer[i, j] = cer_similarity(real_text, results[i][j])
        
    print("\n")

In [ ]:
for i in range(len(results)):
    
    print(f"Real text {i}: {texts[i]}\n")
    
    print(", ".join([f"{mt_jaccard[i, j]:.{precision}f}" for j in range(len(results[0]))]))
    print(", ".join([f"{mt_cer[i, j]:.{precision}f}" for j in range(len(results[0]))]))
        
    print("\n")

In [ ]:
# Select best average case

mean_efectivity_jaccard = np.mean(np.array(mt_jaccard), axis=0)
mean_efectivity_cer = np.mean(np.array(mt_cer), axis=0)

best_fit_jaccard = np.argmax(mean_efectivity_jaccard)
best_fit_cer = np.argmin(mean_efectivity_cer)

best_score_jaccard = mean_efectivity_jaccard[best_fit_jaccard]
best_score_cer = mean_efectivity_cer[best_fit_cer]

print(f"Best fit jaccard: {best_fit_jaccard} with score {best_score_jaccard}")
print(f"Best fit cer: {best_fit_cer} with score {best_score_cer}")


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

# Pyplot 

# Jaccard similarity average case
plt.plot(mean_efectivity_jaccard)
plt.xlabel("Index")
plt.ylabel("Jaccard similarity")
plt.title("Jaccard similarity average case")
plt.show()

# CER average case
plt.plot(mean_efectivity_cer)
plt.xlabel("Index")
plt.ylabel("CER")
plt.title("CER average case")
plt.gca().invert_yaxis()
plt.show()


In [ ]:
import plotly.express as px
import plotly.graph_objects as go

# Generate same plots with plotly
px.line(
    y=mean_efectivity_jaccard, title="Jaccard similarity average case"
).update_layout(yaxis_title="Jaccard similarity").update_layout(
    xaxis_title="pre-processing"
).show()
px.line(
    y=mean_efectivity_cer, title="CER average case"
).update_layout(yaxis_title="CER").update_layout(
    xaxis_title="pre-processing"
).show()

# Post-processing tests

In [ ]:
from src.text_postprocessor import *
import plotly.graph_objects as go


llm = OpenAIModel()

# Extract LLM results
llm_results = [llm.fix_text(result[1]) for result in results]
llm_results = [normalize_text(text) for text in llm_results]

# Calculate Jaccard similarity for LLM results
llm_jaccard = [jaccard_similarity(texts[i], llm_results[i]) for i in range(len(texts))]

# Calculate CER for LLM results
llm_cer = [cer_similarity(texts[i], llm_results[i]) for i in range(len(texts))]

llm = OpenAIModel()

In [ ]:
for i in range(len(results)):
    
    print(f"Real text {i}:\n{texts[i]}\n")
    
    print(f"Tesseract text {i}:\n{results[i][1]}")
    
    print(f"Llm text {i}:\n{llm_results[i]}\n")
    
    print("Distance: ", llm_jaccard[i], " Previous: ", mt_jaccard[i, 1])
    print("Distance: ", llm_cer[i], " Previous: ", mt_cer[i, 1])

In [ ]:
# Create the plot
fig = go.Figure()

# Add trace for LLM results
fig.add_trace(go.Scatter(
    x=list(range(len(llm_jaccard))),
    y=llm_jaccard,
    mode='lines+markers',
    name='LLM Jaccard Similarity'
))

# Add trace for mt_jaccard[i, 1]
fig.add_trace(go.Scatter(
    x=list(range(len(mt_jaccard))),
    y=mt_jaccard[:, 1],
    mode='lines+markers',
    name='Tesseract Jaccard Similarity'
))

# Update layout
fig.update_layout(
    title='LLM Result vs vs just Tesseract',
    xaxis_title='Index',
    yaxis_title='Jaccard Similarity',
    legend_title='Legend'
)

# Show the plot
fig.show()

In [ ]:
# Create the plot
fig = go.Figure()

# Add trace for LLM results
fig.add_trace(go.Scatter(
    x=list(range(len(llm_cer))),
    y=llm_cer,
    mode='lines+markers',
    name='LLM CER Similarity'
))

# Add trace for mt_cer[i, 1]
fig.add_trace(go.Scatter(
    x=list(range(len(mt_cer))),
    y=mt_cer[:, 1],
    mode='lines+markers',
    name='Tesseract CER Similarity'
))

# Update layout
fig.update_layout(
    title='LLM Result vs just Tesseract',
    xaxis_title='Index',
    yaxis_title='CER Similarity',
    legend_title='Legend'
)

# Show the plot
fig.show()